# Google Earth Engine

In [ ]:
# google earth engine should work if installed, but it needs authentication
# making datasets from it with a computer with internet may be preferable to using it while on a compute node
# import ee

# # Trigger the authentication flow.
# ee.Authenticate()

# # Initialize the library.
# ee.Initialize(project='my-project')

# GBIF

In [3]:
from pygbif import species as species
from pygbif import occurrences as occ

In [4]:
splist = ['Cyanocitta stelleri', 'Junco hyemalis', 'Aix sponsa',
  'Ursus americanus', 'Pinus conorta', 'Poa annuus']
keys = [ species.name_backbone(x)['usageKey'] for x in splist ]

In [5]:
out = [ occ.search(taxonKey = x, limit=0)['count'] for x in keys ]

In [6]:
x = dict(zip(splist, out))
sorted(x.items(), key=lambda z:z[1], reverse=True)

[('Junco hyemalis', 14827936),
 ('Aix sponsa', 6498428),
 ('Cyanocitta stelleri', 2756192),
 ('Poa annuus', 741730),
 ('Pinus conorta', 105934),
 ('Ursus americanus', 53946)]

# SatCLIP

In [1]:
import sys
import os

In [2]:
# clone satclip into a folder, use your own username
satclip_path = "/home/mila/g/<username>/scratch/satclip"

In [3]:
sys.path.append(satclip_path)
sys.path.insert(0, "/home/mila/g/<username>/scratch/satclip/satclip")

In [4]:
from satclip.model import SatCLIP

/network/scratch/g/gregory.bell/COMP-685-Fall-2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/network/scratch/g/gregory.bell/COMP-685-Fall-2025/.venv/lib/python3.11/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [5]:
from satclip.location_encoder import LocationEncoder

In [6]:
model = SatCLIP(
    embed_dim=512,
    image_resolution=224, in_channels=13, vision_layers=4, vision_width=768, vision_patch_size=32, # Image encoder
    le_type='sphericalharmonics', pe_type='siren', legendre_polys=10, frequency_num=16, max_radius=360, min_radius=1, harmonics_calculation='analytic'  # Location encoder
)


using vision transformer


In [8]:
import torch

In [9]:
img_batch = torch.randn(32, 13, 224, 224) # Represents a batch of 32 images
loc_batch = torch.randn(32, 2) # Represents the corresponding 32 locations (lon/lat)

In [10]:
with torch.no_grad():
    logits_per_image, logits_per_coord = model(img_batch, loc_batch)
    probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()

# INaturalist

In [11]:
from pyinaturalist import get_observations

In [12]:
observations = get_observations()

In [14]:
observations.keys()

dict_keys(['total_results', 'page', 'per_page', 'results'])

In [17]:
observations['total_results']

312774412

# eBird

In [1]:
# account api key is needed, may be best to load dataset outside computer node.